In [1]:
# Original script by J.Tharsen 03-2025
# modified from www.datacamp.com/tutorial/llama-3-1-rag

# Install libraries (if needed)
#!pip install langchain langchain_community langchain-openai scikit-learn langchain-ollama sentence-transformers

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os, glob
import pandas as pd

# Load documents from local files
docs_list = []
metadata_list = []
source_dir = "./cases"
df = pd.read_csv("./metadata.csv")

for filename in glob.glob(source_dir + "/*.txt"):
    filedata = open(filename, 'r').read()
    docs_list.append(filedata)
    row = df[("./cases/" + df['filename']) == filename]
    if not row.empty:
        # Extract metadata as a dictionary
        metadata = {
            'name': row.iloc[0]['name'],
            'year': row.iloc[0]['year'],
            'legal_issue': row.iloc[0]['legal_issue']
        }
        metadata_list.append(metadata)
    else:
        print(f"Warning: No metadata found for file {filename}")
        metadata_list.append(None)

print(str(len(docs_list)) + " documents loaded.")

USER_AGENT environment variable not set, consider setting it to identify your requests.


16 documents loaded.


In [3]:
# Initialize a text splitter 
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size=500, 
    chunk_overlap=0
)

# Optional arguments for the text_splitter
#    length_function=len,
#    is_separator_regex=False,

# Split the documents into chunks
doc_splits = text_splitter.create_documents(docs_list, metadata_list)

In [4]:
sentences = []
for doc in doc_splits:
    sentences.append(doc.page_content)
len(sentences)

1811

In [5]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings


with open("open_ai_key.txt", "r") as file:
    my_api_key = file.read().strip()
#my_api_key = "your_key_here"
# Create a file in this directory titled "secret_key.txt" and add your key.

# Create embeddings for documents and store them in a vector store
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(openai_api_key=my_api_key),
)
retriever = vectorstore.as_retriever(k=4)

In [6]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are modeled after chief justice John Roberts.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in the style of justice John Roberts.
    Remain brief with a maximum of 7 sentences:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [19]:
# Initialize the LLM with the chosen model, set temperature to 0
llm = ChatOllama(
    model="deepseek-r1:7b",
    temperature=0,
)

In [20]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

In [24]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)

        # Extract content from retrieved documents
        doc_texts = "\n".join([
            f"[METADATA] {doc.metadata}\n[CONTENT] {doc.page_content}" for doc in documents
        ])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer, doc_texts

In [25]:
# Initialize the RAG application
rag_application = RAGApplication(retriever, rag_chain)

In [ ]:
# Run the RAG application
question = "Should Chevron Deference remain?"
answer, doc_texts = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Should Chevron Deference remain?
Answer: <think>
Okay, so I need to figure out whether Chevron Deference should remain based on the provided documents and content. Let me start by understanding what Chevron Deference is. From what I remember, Chevron deference is a legal principle that courts use when interpreting agency regulations. It's named after a Supreme Court case where the court applied broad deference to an agency's interpretation of its own laws.

Looking at the documents provided, there are several metadata entries with different identifiers and years, but they all pertain to the same case: Loper Bright Enterprises v. Raimondo from 2024, dealing with government agencies. The content seems to discuss whether Chevron Deference should remain in effect.

In the first content excerpt, it mentions that under Chevron, ambiguities trigger deference, and both the Government and the dissent argue that Chevron applies even in cases not directly related to an agency's technica